In [ ]:
## imported libraries
from pathlib import Path
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mercury as mr
from roux.lib.io import read_dict,to_dict,read_table,to_table
from roux.lib.sys import get_datetime

from beditor.lib.utils import cols_muts
from beditor.lib.utils import get_src_path
src_path=get_src_path()

In [ ]:
## parameters for testing using papermill
config_path=None
new_run=True
# dbug=True
dbug=False
not_be=True
beditor_score_min=0.8

palette={
    # True:'r',
    True:'#2684FF',
    False:'gray',
    'sg':'#2684FF',
    'none':'lightgray',
}
figsize=[10,8]

force=False

In [ ]:
## inferred parameters
new_run=config_path is None

In [ ]:
# control app with App class
app = mr.App(
    title="beditor", 
    # description="Showcase of Mercury Widgets",
    show_code = dbug,
    # show_code = True,
    show_prompt=dbug,
    # stop_on_error=True,
    # continuous_update=not dbug,
    continuous_update=False,
)

In [ ]:
inew_run = mr.Checkbox(value=new_run, label="New run")
# read checkbox value
new_run=inew_run.value

In [90]:
# open existing using the output directory    
# dv="path/to/parameters.yaml"
# iconfig_path = mr.Text(value=dv, label="Load parameters", rows=1)
# config_path=iconfig_path.value
# if config_path==dv:
#     sys.exit(0)
iconfig_file = mr.File(label="Load config", max_file_size="10MB",hidden=new_run)
if not new_run:
    if config_path is None:
        config_path=iconfig_file.filepath
    if config_path is None:
        mr.Stop()
    else:
        if dbug:
            print(config_path,Path(config_path).exists())
        if Path(config_path).exists():
            cfg=read_dict(config_path,fmt='yaml')
            if dbug:
                print(cfg)
        else:
            print("Config not loaded")
            mr.Stop()
else:
    parameters={}

mercury.File

{'run': {'dbug': False, 'force': True, 'genome_path': 'inputs/dna.fa', 'gtf_path': 'inputs/ann.gtf', 'igv_path_prefix': None, 'input_path': 'inputs/mutations/protein//points.tsv', 'method': 'Cas12a-BE', 'not_be': False, 'output_path': 'outputs/protein/points//output.tsv', 'protein_path': 'inputs/Protein.fa', 'release': None, 'search_window': None, 'species_name': None, 'test': False, 'threads': 10, 'transcript_path': 'inputs/RNA.fa', 'verbose': False, 'wd_path': '/mnt/d/Documents/code/beditor/examples'}}


In [91]:
## workflow related widgets

In [ ]:
_ = mr.Note(text='<a href="https://github.com/rraadd88/beditor?tab=readme-ov-file#gui-mode" target="_blank">ⓘ</a>')

In [ ]:
## genome
if new_run:
    from pyensembl import species
    species_names=[f"{k.capitalize()} ({getattr(species,k).latin_name.capitalize().replace('_',' ')})" for k in dir(species) if isinstance(getattr(species,k),species.Species)]
    if dbug:
        print(species_names)

In [ ]:
if new_run:
    ispecies = mr.Select(
        label="Species",
        # value=[s for s in species_names if 'human' in s.lower()][0],
        value='',
        choices=['']+species_names,
        hidden=not new_run,
        )

In [ ]:
if new_run:
    if ispecies.value=='':
        mr.Stop()

In [ ]:
if new_run:
    # access selected values
    parameters['species']=ispecies.value.split(' (')[0].lower()
    cfg_species=getattr(species,parameters['species'])
    reference_assemblies=list(cfg_species.reference_assemblies.keys())[::-1]
    reference_assemblies=list(set(reference_assemblies) - set(['GRCh37','NCBI36']))
    if dbug:
        print(reference_assemblies)

In [ ]:
if new_run:
    iassembly = mr.Select(
        label="Genome assembly",
        value=reference_assemblies[0],
        choices=reference_assemblies,
        hidden=not new_run,
        )

In [ ]:
if new_run:
    assembly=iassembly.value
    releases=cfg_species.reference_assemblies[assembly]
    releases=list(map(str,list(range(releases[0],releases[1]+1,1))))[::-1]
    if dbug:
        print(releases)

In [ ]:
if new_run:
    irelease = mr.Select(
        label="Ensembl release",
        value=releases[0],
        choices=releases,
        hidden=not new_run,
        )
    parameters['ensembl_release']=irelease.value

In [ ]:
im_file = mr.File(
    label="Load mutations", max_file_size="100MB",
    disabled=not new_run,
    hidden=not new_run,
    )
if new_run:
    parameters['mutations_path']=im_file.filepath
    if parameters['mutations_path'] is None:
        mr.Stop()

In [ ]:
if new_run:
    ibe = mr.Checkbox(value=not_be, label="Base editing")
    # read checkbox value
    parameters['not_be']=ibe.value

In [ ]:
editors=sorted(read_table(f"{src_path}/data/methods.tsv")['method'].unique().tolist())

In [ ]:
ied = mr.Select(
    label="Editor",
    value='',
    choices=['']+editors,
    hidden=not new_run,
    )

In [ ]:
if new_run:
    parameters['editor']=ied.value
    if parameters['editor']=='':
        mr.Stop()

In [ ]:
if new_run:
    from roux.lib.sys import get_datetime
    # output folder.
    # ioutput_dir_path = mr.OutputDir()
    # output_dir_path=ioutput_dir_path.path
    
    parameters['output_dir_path']=f".beditor_gui_sessions/{get_datetime()}/"
    if dbug:
        print(parameters['output_dir_path'])

In [ ]:
## run
if new_run:
    # to_dict(parameters,f"{output_dir_path}/inputs.yaml")
    try:
        from beditor.run import cli
        cli(
            **parameters,
            threads = 1,
            verbose="CRITICAL",
            dbug = dbug,
            skip = ['Visual'],
        )
    except:
        print(f"Check the temporaty outputs stored at: {parameters['output_dir_path']}")
        mr.Stop()
    cfg=read_dict(f"{parameters['output_dir_path']}/config.yaml",fmt='yaml')
    del parameters
else:
    if not 'wd_path' in cfg['run']:
        print("Load the the config from the output directory.")
        mr.Stop()
        # raise mr.StopExecution("Load the the config from the output directory.")
not_be=cfg['run']['not_be']
wd_path=cfg['run']['wd_path']
if dbug:
    print(f"wd_path='{cfg['run']['wd_path']}'")
output_dir_path=Path(cfg['run']['output_path']).parent
if dbug:
    print(f"output_dir_path='{output_dir_path}'")

In [ ]:
## Designed sgRNAs
# wd_path='/mnt/d/Documents/code/beditor/examples'
# output_dir_path='outputs/protein/regions'

In [ ]:
if not wd_path is None:
    import os 
    os.chdir(wd_path)

In [ ]:
## outputs
output_paths=read_dict(f"{output_dir_path}/output_paths.yaml")
# cfg=read_dict(f"{output_dir_path}/config.yaml")
cfg_method=read_dict(Path(output_paths['output']).parent.as_posix()+'/00_inputs/pam.yaml')
if dbug:
    print(cfg_method)

## **Loaded config**

In [ ]:
pd.DataFrame(cfg).dropna().T.sort_index(axis=1).set_index(['method','input_path'])

In [ ]:
df0=read_table(output_paths['input_mutations'])
if dbug:
    print(df0.head(1))

In [ ]:
df1=read_table(output_paths['output_full']).rd.assert_no_dups()
if dbug:
    print(df1.head(1))

In [ ]:
stats=dict(
    input_mutations_unique=len(df0.loc[:,cols_muts].drop_duplicates()),
    input_mutations_targeted=len(df1.loc[:,cols_muts].drop_duplicates()),
    sgs_designed=df1['guide sequence'].nunique(),
)
if 'protein id' in df0:
    stats['input_proteins_unique']=df0['protein id'].nunique()
    stats['input_proteins_targeted']=df1['protein id'].nunique()
if 'gene id' in df0:
    stats['input_genes_unique']=df0['gene id'].nunique()
    stats['input_genes_targeted']=df1['gene id'].nunique()
# mr.Md(f"{df1['guide sequence'].nunique()} sgRNAs are designed for {sgs_count}/{len(df0)} input mutations")

In [ ]:
# #### By offtarget aligned genes
if not 'aligned genes count' in df1:
    df2=df1.assign(**{
        'aligned genes count':lambda df: df['aligned genes'].apply(lambda x: x.count(';')+1),
        'polyT stretch length': lambda df: df['polyT stretch length'].fillna(0),
    })
else:
    df2=df1.copy()
if dbug:
    print(df2.head(1))

In [ ]:
# more boxes in one row
mr.NumberBox(
    ([
        mr.NumberBox(data=stats['sgs_designed'], title="sgRNAs designed"),
        mr.NumberBox(data=f"{stats['input_mutations_targeted']}/{stats['input_mutations_unique']}", title="loci"),
    ]
    +[
        mr.NumberBox(data=f"{stats['input_genes_targeted']}/{stats['input_genes_unique']}", title="genes"),
    ] if 'input_genes_unique' in stats else []    
    )
)

### **Scores of sgRNAs**

In [ ]:
_ = mr.Note(text="#### Filtering")

In [ ]:
from beditor.lib.viz import get_plot_inputs
dfs=get_plot_inputs(
    df2,
    )

In [ ]:
## defaults
cutoffs={
    'Off-target alignments':
        [
            dfs['Off-target alignments'].index.min(),
            dfs['Off-target alignments'].index.max(),
        ],
    "Genes targeted by sgRNA":
        [
            dfs['Genes targeted by sgRNA'].index.min(),
            dfs['Genes targeted by sgRNA'].index.max(),
        ],
    "PolyT stretch":
        [
           dfs['PolyT stretch'].index.min(), 
           dfs['PolyT stretch'].index.max(), 
        ],
    "beditor score":
        [
           dfs['beditor score'].min(), 
           dfs['beditor score'].max(),             
        ],
        }
# cutoffs

In [ ]:
k='beditor score'
inputs = mr.Range(
    value=[i*100 for i in cutoffs[k]],
    # min=cutoffs[k][0]*100,
    # max=cutoffs[k][1]*100,
    min=0,
    max=100,
    label=k+('*' if not_be else ''),
    step=1,
    hidden=len(dfs[k])==0,
)
cutoffs[k]=[i/100 for i in inputs.value]

In [ ]:
k='Off-target alignments'
inputs = mr.Range(
    value=cutoffs[k],
    min=cutoffs[k][0],
    max=cutoffs[k][1],
    label=k,
    step=1,
    hidden=len(dfs[k])==0,
)
cutoffs[k]=inputs.value

In [ ]:
k='Genes targeted by sgRNA'
inputs = mr.Range(
    value=cutoffs[k],
    min=cutoffs[k][0],
    max=cutoffs[k][1],
    label=k,
    step=1,
    hidden=len(dfs[k])==0,
)
cutoffs[k]=inputs.value

In [ ]:
k='PolyT stretch'
inputs = mr.Range(
    value=cutoffs[k],
    min=cutoffs[k][0],
    max=cutoffs[k][1],
    label=k,
    step=1,
    hidden=len(dfs[k])==0,
)
cutoffs[k]=inputs.value

In [ ]:
# cutoffs

In [ ]:
from beditor.lib.viz import plot_library_stats
axs=plot_library_stats(
    dfs,
    not_be=not_be,
    # cutoffs=cutoffs,
    palette=palette,
    figsize=[figsize[0],figsize[0]/4.5],
    )

In [ ]:
import mercury as mr
# add checkbox
ishow_more = mr.Checkbox(value=False, label="Show more")
show_more=ishow_more.value

In [ ]:
if dbug or show_more:
    mr.Md("### Scores of alignments")
    df_=read_table(output_paths['alignments_mapped'])
    df_.head(1)

    import matplotlib.pyplot as plt
    fig,axs=plt.subplots(1,4,figsize=[figsize[0],figsize[0]/4],sharey=False)

    df_=(df_
    # .query(expr="`offtarget`==True")
    .assign(
        **{
            'mismatches count': lambda df: df['alignment'].apply(lambda x: x.count('.')),
            'mismatches position': lambda df: df['alignment'].apply(lambda x: x.index('.') if '.' in x else np.nan),
        }
    )
    )
    for c,ax in zip(['mismatches count','mismatches position'],axs[:2]):
        df1_=df_.dropna(subset=[c]).astype({c:int})
        if len(df1_)==0:
            continue
        ax=df1_.boxplot(by=c,column='penalty mismatches',ax=ax)
        ax.grid(False)
        _=ax.set(
            ylabel='penalty score',
            title=None,
            )
        if c=='mismatches position':
            ax.xaxis.set_major_locator(plt.MaxNLocator(5))
            # ax.set(xticks=)
    for c,ax in zip(['alignments','aligned genes count'],axs[2:]):
        ax=df2.dropna(subset=[c]).astype({c:int}).query(f'`{c}`<=5').boxplot(by=c,column='score',ax=ax)
        ax.grid(False)
        _=ax.set(
            ylabel='beditor score'+('*' if not_be else ''),
            title=None,
            )
    plt.suptitle(None)
    plt.tight_layout()

## **After filtering**

In [ ]:
# {'Off-target alignments': [0, 12],
#  'Genes targeted by sgRNA': [1, 7],
#  'PolyT stretch': [1, 9],
#  'beditor score': [0.19, 0.7]}
if dbug:
    cutoffs

In [ ]:
stats['filtering']={}

# for k in cutoffs:
k='Off-target alignments'
df3=(df2
    .log('guide sequence')
    .groupby(by='guide sequence')
        .filter(lambda df: all((df['offtargets']>=cutoffs[k][0]) & (df['offtargets']<=cutoffs[k][1])))
    .log('guide sequence')
)
stats['filtering']['offtarget']=dict(
    sgs=df3['guide sequence'].nunique()
    )

k='Genes targeted by sgRNA'
df3=(df3
    .log('guide sequence')
    .query(expr=f"`aligned genes count` >= {cutoffs[k][0]} and `aligned genes count` <= {cutoffs[k][1]}")
    .log('guide sequence')
)
stats['filtering'][k]=dict(
    sgs=df3['guide sequence'].nunique()
    )

k='PolyT stretch'
df3=(df3
    .log('guide sequence')
    .query(expr=f"`polyT stretch length` >= {cutoffs[k][0]} & `polyT stretch length` <= {cutoffs[k][1]}")
    .log('guide sequence')
)
stats['filtering'][k]=dict(
    sgs=df3['guide sequence'].nunique()
    )

k='beditor score'
df3=(df3
    .log('guide sequence')
    .query(expr=f"`score` >= {cutoffs[k][0]} & `score` <= {cutoffs[k][1]}")
    .log('guide sequence')
)
stats['filtering'][k]=dict(
    sgs=df3['guide sequence'].nunique()
    )

In [ ]:
if len(df3)==len(df2):
    mr.Stop()

In [ ]:
cols_input=df3.columns.tolist()[:df3.columns.tolist().index('chrom')]#+['target position']

In [ ]:
df3=df3.assign(
    **{
        # 'target location':lambda df: df.apply(lambda x: f"{x['chrom']}:{int(x['start'])}-{int(x['end'])}{x['strand']}",axis=1),
        'target locus':lambda df: df.apply(lambda x: ','.join([str(x[c]) for c in cols_input])+f" ({x['target location']})",axis=1),
    }
    )

In [ ]:
## error here
sg_per_target_max=df3.groupby(cols_input+['target locus'])['guide sequence'].nunique().min()
if sg_per_target_max>1:
    isg_per_target_max= mr.Slider(
        value=sg_per_target_max,
        min=1,
        max=sg_per_target_max,
        label="Top sgRNAs per target",
        step=1,
        # hidden=len(data.index.tolist())==0,
    )
    sg_per_target_max=isg_per_target_max.value

In [ ]:
if sg_per_target_max!=sg_per_target_max:
    df4=(df3
        .groupby(cols_input+['target locus'],as_index=False)
            .apply(lambda df: df.sort_values(['score','polyT stretch length','offtargets'],ascending=[False,True,True]).head(sg_per_target_max))
        .reset_index(drop=True)
    )
else:
    df4=df3.sort_values(['score','polyT stretch length','offtargets'],ascending=[False,True,True])
if dbug:
    print(df4.head(1))
del df3

In [ ]:
stats['sgs_remained']=df4['guide sequence'].nunique()
stats['input_mutations_remained']=len(df4.loc[:,cols_muts].drop_duplicates())
if 'protein id' in df0:
    stats['input_proteins_remained']=df4['protein id'].nunique()
if 'gene id' in df0:
    stats['input_genes_remained']=df4['gene id'].nunique()
if dbug:
    print(stats)

In [ ]:
# more boxes in one row
mr.NumberBox(
    ([
        mr.NumberBox(data=stats['sgs_remained'], title="sgRNAs remained"),
        mr.NumberBox(data=f"{stats['input_mutations_remained']}/{stats['input_mutations_unique']}", title="loci"),
    ]
    +[
        mr.NumberBox(data=f"{stats['input_genes_remained']}/{stats['input_genes_unique']}", title="genes"),
    ] if 'input_genes_unique' in stats else []    
    )
)

In [ ]:
axs=plot_library_stats(
    dfs=get_plot_inputs(
        df4,
    ),
    not_be=not_be,
    cutoffs=cutoffs,
    palette=palette,
    figsize=[figsize[0],figsize[0]/4.5],
    )

### **Nucleotide composition**

In [ ]:
## nt composition
from beditor.lib.viz import plot_ntcompos
fig,ax=plt.subplots(figsize=[figsize[0],2])
_=plot_ntcompos(
    seqs=df4['guide+PAM sequence'].tolist(),
    pam_pos=cfg_method['PAM position'],
    pam_len=len(cfg_method['PAM']),
    window=None if not_be else (cfg_method['distance of mutation from PAM: minimum'],cfg_method['distance of mutation from PAM: maximum']),
    color_pam=palette['none'],
    ax=ax,
    )

In [ ]:
# get cytobands
# else, get the bounds from the gtf

In [ ]:
gtf_path=cfg['run']['gtf_path']
genome_path=cfg['run']['genome_path']

In [ ]:
if gtf_path is None:
    # cfg_species=read_dict(Path(cfg['run']['output_path']).parent.as_posix()+'/00_inputs/species.yaml')
    cfg_species=read_dict(output_paths['input_species'])
    gtf_path=cfg_species['gtf_path']

In [ ]:
if Path(gtf_path).name.startswith("Homo_sapiens.GRCh38"):
    #todo get the cytoBandIdeo file for other species
    cytoBandIdeo_path="https://igv-genepattern-org.s3.amazonaws.com/genomes/hg38/cytoBandIdeo.txt.gz"
    cytobands_path=f"{Path(output_paths['output']).parent.as_posix()}/06_viz/cytobands.tsv"
    if not Path(cytobands_path).exists():
        from beditor.lib.io import read_cytobands
        df01=read_cytobands(cytoBandIdeo_path)
        to_table(df01,cytobands_path)
    else:
        df01=read_table(cytobands_path)
else:
    df01=(
            pd.read_csv(genome_path+'.fai',sep='\t',names=['chrom','length','byte index','bases per line','bytes per line'])
            .loc[:,['chrom','length']]
            .astype({'chrom':str,'length': int})
            .assign(
                start=1,
                arm='arm',
                end=lambda df: df['length']
            )
    )

In [ ]:
col_chrom='chromosome'
cytobands=(df01
    .rename(
        columns={
            'chrom':col_chrom,
            "chrom arm":f"{col_chrom} arm",
        },
        )
    # .assign(
    #     start = 1,
    #     end   = lambda df: df['length'],
    #     )
    # .drop(['length'],axis=1)
    )
if dbug:
    print(cytobands.head(1))

## **Inspect**

In [ ]:
_=mr.Note("### Inspect")

In [ ]:
ishow_offtargets_only = mr.Checkbox(value=False, label="Show off-targets")
# read checkbox value
show_offtargets_only=ishow_offtargets_only.value

In [ ]:
ishow_coedits_only = mr.Checkbox(value=False, label="Show coedits")
# read checkbox value
show_coedits_only=ishow_coedits_only.value

In [ ]:
## select the target

In [ ]:
#alignments_mapped
df02=read_table(output_paths['alignments_mapped'])
# ## back-compatibility
if "guide location" in df02:
    df02=df02.rename(
        columns={
        "guide location":"guide locus",
        "aligned location":"aligned locus",
        },
    )
if dbug:
    print(df02.columns.tolist())
df02=(df02
    .assign(
        **{
            'sgRNA sequence (locus)': lambda df: df.apply(lambda x: f"{x['guide sequence']} ({x['guide locus']})",axis=1),
        }
    )
    )

if dbug:
    print(df02.head(1))

In [ ]:
expr_filter_targets=None
if show_offtargets_only and not show_coedits_only:
    expr_filter_targets=f"`offtargets`!=0"
elif not show_offtargets_only and show_coedits_only:
    expr_filter_targets=f"`coedits`!=''"
elif show_offtargets_only and show_coedits_only:
    expr_filter_targets=f"`offtargets`!=0 and `coedits`!=''"
    
if not expr_filter_targets is None:
    loci=sorted(df4.log.query(expr=expr_filter_targets)['target locus'].unique().tolist())
else:
    loci=sorted(df4['target locus'].unique().tolist())
if dbug:
    print(len(loci))

In [ ]:
if len(loci)==0:
    mr.Stop()

In [ ]:
iloc = mr.Select(
    label="Target ("+','.join([str(s) for s in cols_input])+" (locus))",
    value=loci[0],
    choices=loci,
    )
loc=iloc.value

In [ ]:
if "guide location" in df4:
    df4=df4.rename(
        columns={
        "guide location":"guide locus",
        "aligned location":"aligned locus",
        },
    )
df5=(df4
    .query(f"`target locus`=='{loc}'")
    .assign(
        **{
            'sgRNA sequence (locus)': lambda df: df.apply(lambda x: f"{x['guide sequence']} ({x['guide locus']})",axis=1),
        },
    )
    )
# if dbug:
    # print(df5.T)
# df5['offtargets'].value_counts()

In [ ]:
if not expr_filter_targets is None:
    df_=df5.log.query(expr=expr_filter_targets)
else:
    df_=df5.copy()

In [ ]:
sgloci=df_['sgRNA sequence (locus)'].tolist()
if len(sgloci)==0:
    mr.Stop()
isgloc = mr.Select(
    label="sgRNA",
    value=sgloci[0],
    choices=sgloci,
    )
sgloc=isgloc.value

In [ ]:
if dbug:
    print(sgloc)

In [ ]:
df_=df_.query(expr=f"`sgRNA sequence (locus)`=='{sgloc}'")
# df_

In [ ]:
# ## visualise
# dna feature viewer zoom on the target

In [ ]:
aligns=df02.query(expr=f"`sgRNA sequence (locus)`=='{sgloc}'")

In [ ]:
mr.Md(f"### Target: `{loc}`")
mr.Md(f"### sgRNA: `{sgloc}`")

In [ ]:
x=aligns.query("`offtarget`==False")
assert len(x)==1, f"expected 1 row but found {len(x)}, possible duplicates"
x=x.iloc[0,:].to_dict()
if dbug:
    print(x)

In [ ]:
# %run ../beditor/lib/viz.py
from beditor.lib.viz import plot_ontarget
_=plot_ontarget(
    guide_loc=x['guide locus'],
    # guide_strand=aligns.query('`offtarget`==False').iloc[0,:]['aligned strand'],
    pam_pos=cfg_method['PAM position'],
    pam_len=len(cfg_method['PAM']),
    guidepam_seq=x['guide+PAM sequence'],
    window=None if not_be else (cfg_method['distance of mutation from PAM: minimum'],cfg_method['distance of mutation from PAM: maximum']),
    verbose=dbug,
    kws_sg=dict(color=palette['sg']),
    )

In [ ]:
# chrov to show the offtargets
# show the table for the input sgRNA

In [ ]:
if dbug:
    # print(df02.columns.tolist())
    # print(data.dropna(axis=1).to_dict())
    print(cytobands.to_dict())

#### **Genome-wide off-targets (if any)**

In [ ]:
data=(aligns
    .assign(
        chromosome=lambda df: df['guide locus'].apply(lambda x: x.split(':')[0]),
        label=lambda df: df['offtarget'].map({False:'On',True:'Off'})
    )
    .loc[:,['chromosome','aligned start','label','score per alignment']]
    .astype({
        'chromosome':str,
        "aligned start": int,
        "score per alignment": float,
    }
    )
)
if dbug: 
    print(data)

In [ ]:
from chrov.viz.figure import plot_with_genome
# %run ../../chrov/chrov/viz/figure.py
coly='score per alignment'
axs=plot_with_genome(
    data=data.copy(),
    cytobands=cytobands,
    kind='stem',
    colx='aligned start',
    coly=coly,
    col_label='label',
    # coffy=0.9,
    xkind='loci',
    chrom_y=0,
    arc=True,
    va='center',
    off=0.8,
    offy=0.4,
    figsize=[4,4],
    kws_seaborn=dict(linefmt=palette['sg']),
    kws_annot_chroms={},
    kws_annot_labels=dict(loc='in'),
    )
axs['data'].set(ylim=[0,1])
for t in axs['data'].get_children():
    if isinstance(t,plt.Text):
        if t.get_text()==coly:
            # print(t.get_position())
            t.set_text(f"\n\n{t.get_text()}")
            t.set_x(0)
            break

#### **Alignments of the sgRNA**

In [ ]:
# data.columns.tolist()
aligns.loc[:,
[
 # 'guide sequence',
 # 'guide locus',
 # 'aligned NM',
 # 'aligned X0',
 # 'aligned X1',
 # 'aligned XG',
 # 'aligned XM',
 # 'aligned XO',
 # 'aligned XT',
 # 'aligned chrom',
 # 'aligned end',
 # 'aligned mapq',
 'offtarget',
 'alignment is perfect',
 'aligned locus',
 # 'aligned start',
 # 'aligned strand',
 # 'alignments',
 # 'aligned PAM start',
 # 'aligned PAM end',
 # 'aligned PAM',
 'aligned PAM valid',
 'aligned genes',
 # 'aligned genic',
 # 'alignment',
 # 'penalty mismatches',
 'score per alignment',
 'sgRNA sequence (locus)',
 'aligned locus',
 'aligned sequence',
 # 'chromosome',
 # 'label',
    
]
        ].set_index('sgRNA sequence (locus)')

In [ ]:
#_ = mr.Note(text="#### Outputs")
isave = mr.Button(label="Save the filtered library", style="primary")
save=isave.clicked
if save:
 #   mr.Md("## Outputs")
    # mr.Md(f"#### The QC filtered library is save at    : "+to_table(df3,f"{output_dir_path}/filtered/{get_datetime()}_qc.tsv"))
    # mr.Md(f"#### The library with top sgRNAs is save at: "+to_table(df4,f"{output_dir_path}/filtered/{get_datetime()}_top.tsv"))
    mr.Md(f"#### The filtered library is save at: "+to_table(df4,f"{output_dir_path}/filtered/{get_datetime()}.tsv"))    

In [ ]:
if dbug:
    open(f"{output_dir_path}/gui.log",'a').write('')